This notebook takes the working code from AZoneDissolve_Texas.ipynb and integrates the AE Merge polygons which touch an Azone. In short, any AE split polygon that touches an Azone will need to be removed from the AE_Zone split dataframe and added to the AZone dataframe. The new verision of the AE Split dataframe will then be ready for the AE triangulation after the AZone triangualtion

In [216]:
import os
import glob
import math
from processing_tools import *
from shapely.geometry import MultiPoint
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.ops import triangulate
from shapely.ops import unary_union

In [232]:
""" Read and collect AZones for each TX county into single dataframe, then dissolve """

state_path = 'D:\GitHub_Repos\GIS\BFEs\Workspace\WorkingScripts_v1014_2022\Preprocessing\CO_FZ'
elevVrt = 'D:\GitHub_Repos\GIS\BFEs\Workspace\WorkingScripts_v1014_2022\Preprocessing\OtherZone_testing\Dallas_1m.vrt"'

# Read in raw County Flood for A zone
for fi in glob.glob(os.path.join(state_path, '*08013.shp')):
    print('Reading', fi)
    fz = gpd.read_file(fi)
# Locate A Zone
a = fz.loc[fz['ZONE'] == 'A', ['FIPS', 'geometry']]

# Read in AE* Split File (from preprocessing phase)
ae = gpd.read_file(r'D:\GitHub_Repos\GIS\BFEs\Workspace\WorkingScripts_v1014_2022\Preprocessing\08013_FloodZone_split.shp')
ae_geom = ae[['geometry']]

# Read in BFE file (from preprocessing phase)
bfe = gpd.read_file(r'D:\GitHub_Repos\GIS\BFEs\Workspace\WorkingScripts_v1014_2022\Preprocessing\BFE_08013.shp')

# Locate Flood Zones that only have one BFEs (should be ends). This will then join to Azone


# Dissolve and explode
a = a.dissolve()
a = a.explode()
a = a.reset_index(0, drop=True)
# determine correct utm code
crs = utm_code(a)

Reading D:\GitHub_Repos\GIS\BFEs\Workspace\WorkingScripts_v1014_2022\Preprocessing\CO_FZ\FM_08013.shp


In [233]:
ae_join = ae_geom.sjoin(bfe)
ae_join

,geometry,index_right,ELEV
0,"POLYGON ((-105.51918 39.95856, -105.51918 39.9...",4174,8321.0
1,"POLYGON ((-105.51918 39.95856, -105.51919 39.9...",4174,8321.0
1,"POLYGON ((-105.51918 39.95856, -105.51919 39.9...",4173,8316.0
4,"POLYGON ((-105.51871 39.95871, -105.51882 39.9...",4173,8316.0
2,"POLYGON ((-105.51904 39.96504, -105.51904 39.9...",4172,8306.0
...,...,...,...
4236,"POLYGON ((-105.05550 40.06421, -105.05551 40.0...",3522,4997.0
4236,"POLYGON ((-105.05550 40.06421, -105.05551 40.0...",3523,4998.0
4239,"POLYGON ((-105.05551 40.06354, -105.05551 40.0...",3523,4998.0
4237,"POLYGON ((-105.05701 40.14410, -105.05702 40.1...",2018,4904.0


In [234]:
vals = ae_join.index.value_counts()
vals

1923    31
1646    28
2392    28
1597    14
3875    14
        ..
3227     1
3241     1
3242     1
3248     1
4240     1
Length: 4241, dtype: int64

In [235]:
ae_single = ae_join.loc[ae_join.index.isin(vals.index[vals.lt(2)])]
ae_single

,geometry,index_right,ELEV
0,"POLYGON ((-105.51918 39.95856, -105.51918 39.9...",4174,8321.0
2,"POLYGON ((-105.51904 39.96504, -105.51904 39.9...",4172,8306.0
13,"POLYGON ((-105.51685 40.06841, -105.51674 40.0...",1605,9245.0
80,"POLYGON ((-105.50470 39.96170, -105.50470 39.9...",1345,8190.0
87,"POLYGON ((-105.50305 40.13234, -105.50319 40.1...",721,8509.0
...,...,...,...
4204,"POLYGON ((-105.06303 40.08857, -105.06284 40.0...",3242,4950.0
4238,"POLYGON ((-105.05669 40.09374, -105.05547 40.0...",1525,4951.0
4233,"POLYGON ((-105.05698 40.00398, -105.05797 40.0...",1330,5098.5
4239,"POLYGON ((-105.05551 40.06354, -105.05551 40.0...",3523,4998.0


In [236]:
ae_single = ae_single[['geometry']]
ae_single

,geometry
0,"POLYGON ((-105.51918 39.95856, -105.51918 39.9..."
2,"POLYGON ((-105.51904 39.96504, -105.51904 39.9..."
13,"POLYGON ((-105.51685 40.06841, -105.51674 40.0..."
80,"POLYGON ((-105.50470 39.96170, -105.50470 39.9..."
87,"POLYGON ((-105.50305 40.13234, -105.50319 40.1..."
...,...
4204,"POLYGON ((-105.06303 40.08857, -105.06284 40.0..."
4238,"POLYGON ((-105.05669 40.09374, -105.05547 40.0..."
4233,"POLYGON ((-105.05698 40.00398, -105.05797 40.0..."
4239,"POLYGON ((-105.05551 40.06354, -105.05551 40.0..."


In [237]:
# Spatial join to find those that touch
ajoin = a.sjoin(ae_single)[['geometry', 'index_right']]
ajoin

,geometry,index_right
0,"POLYGON ((-105.49186 39.96219, -105.49189 39.9...",80
0,"POLYGON ((-105.49186 39.96219, -105.49189 39.9...",158
2,"POLYGON ((-105.29545 40.04334, -105.29546 40.0...",1308
5,"POLYGON ((-105.22598 39.92522, -105.22590 39.9...",2740
9,"POLYGON ((-105.19577 39.99882, -105.19578 39.9...",2982
11,"POLYGON ((-105.28584 40.02689, -105.28585 40.0...",1446
11,"POLYGON ((-105.28584 40.02689, -105.28585 40.0...",1451
14,"POLYGON ((-105.18888 40.17915, -105.18952 40.1...",2379
14,"POLYGON ((-105.18888 40.17915, -105.18952 40.1...",3200
16,"POLYGON ((-105.28578 40.22724, -105.28617 40.2...",1489


In [238]:
# Another spatial join to obtain AE end geometries. 
ajoin_geoms = ajoin.merge(ae_single, left_on='index_right', right_index=True, how='left')
ajoin_geoms

,geometry_x,index_right,geometry_y
0,"POLYGON ((-105.49186 39.96219, -105.49189 39.9...",80,"POLYGON ((-105.50470 39.96170, -105.50470 39.9..."
0,"POLYGON ((-105.49186 39.96219, -105.49189 39.9...",158,"POLYGON ((-105.48201 39.96589, -105.48201 39.9..."
2,"POLYGON ((-105.29545 40.04334, -105.29546 40.0...",1308,"POLYGON ((-105.29469 40.04289, -105.29503 40.0..."
5,"POLYGON ((-105.22598 39.92522, -105.22590 39.9...",2740,"POLYGON ((-105.22555 39.92637, -105.22555 39.9..."
9,"POLYGON ((-105.19577 39.99882, -105.19578 39.9...",2982,"POLYGON ((-105.19537 39.99944, -105.19515 39.9..."
11,"POLYGON ((-105.28584 40.02689, -105.28585 40.0...",1446,"POLYGON ((-105.28584 40.02688, -105.28586 40.0..."
11,"POLYGON ((-105.28584 40.02689, -105.28585 40.0...",1451,"POLYGON ((-105.28584 40.02712, -105.28591 40.0..."
14,"POLYGON ((-105.18888 40.17915, -105.18952 40.1...",2379,"POLYGON ((-105.24891 40.21126, -105.24890 40.2..."
14,"POLYGON ((-105.18888 40.17915, -105.18952 40.1...",3200,"POLYGON ((-105.16837 40.17380, -105.16841 40.1..."
16,"POLYGON ((-105.28578 40.22724, -105.28617 40.2...",1489,"POLYGON ((-105.28440 40.22712, -105.28440 40.2..."


In [239]:
a_gb = ajoin_geoms.groupby(level=0)['geometry_y'].apply(list)
a_gb

0     [POLYGON ((-105.504699 39.96169900000001, -105...
2     [POLYGON ((-105.294688 40.042893, -105.295027 ...
5     [POLYGON ((-105.225554 39.926365000000004, -10...
9     [POLYGON ((-105.19537 39.99943999999999, -105....
11    [POLYGON ((-105.285845 40.02688199999999, -105...
14    [POLYGON ((-105.248906 40.211259000000005, -10...
16    [POLYGON ((-105.284402 40.22711600000001, -105...
19    [POLYGON ((-105.172974 39.94782, -105.17343100...
24    [POLYGON ((-105.17958470623196 40.135513908182...
Name: geometry_y, dtype: object

In [240]:
a_merge = a.merge(a_gb, left_index=True, right_index=True)
a_merge

,FIPS,geometry,geometry_y
0,08013,"POLYGON ((-105.49186 39.96219, -105.49189 39.9...","[POLYGON ((-105.504699 39.96169900000001, -105..."
2,08013,"POLYGON ((-105.29545 40.04334, -105.29546 40.0...","[POLYGON ((-105.294688 40.042893, -105.295027 ..."
5,08013,"POLYGON ((-105.22598 39.92522, -105.22590 39.9...","[POLYGON ((-105.225554 39.926365000000004, -10..."
9,08013,"POLYGON ((-105.19577 39.99882, -105.19578 39.9...","[POLYGON ((-105.19537 39.99943999999999, -105...."
11,08013,"POLYGON ((-105.28584 40.02689, -105.28585 40.0...","[POLYGON ((-105.285845 40.02688199999999, -105..."
14,08013,"POLYGON ((-105.18888 40.17915, -105.18952 40.1...","[POLYGON ((-105.248906 40.211259000000005, -10..."
16,08013,"POLYGON ((-105.28578 40.22724, -105.28617 40.2...","[POLYGON ((-105.284402 40.22711600000001, -105..."
19,08013,"POLYGON ((-105.17234 39.94832, -105.17242 39.9...","[POLYGON ((-105.172974 39.94782, -105.17343100..."
24,08013,"POLYGON ((-105.17956 40.13560, -105.17935 40.1...",[POLYGON ((-105.17958470623196 40.135513908182...


In [242]:
# union polygons
a_merge['new_geom'] = a_merge.apply(lambda x: unary_union([x.geometry] + x.geometry_y), axis=1)
a_merge.head()

,FIPS,geometry,geometry_y,new_geom
0,08013,"POLYGON ((-105.49186 39.96219, -105.49189 39.9...","[POLYGON ((-105.504699 39.96169900000001, -105...","MULTIPOLYGON (((-105.50470 39.96170, -105.5046..."
2,08013,"POLYGON ((-105.29545 40.04334, -105.29546 40.0...","[POLYGON ((-105.294688 40.042893, -105.295027 ...","POLYGON ((-105.29503 40.04274, -105.29504 40.0..."
5,08013,"POLYGON ((-105.22598 39.92522, -105.22590 39.9...","[POLYGON ((-105.225554 39.926365000000004, -10...","POLYGON ((-105.22584 39.92512, -105.22596 39.9..."
9,08013,"POLYGON ((-105.19577 39.99882, -105.19578 39.9...","[POLYGON ((-105.19537 39.99943999999999, -105....","POLYGON ((-105.19578 39.99881, -105.19585 39.9..."
11,08013,"POLYGON ((-105.28584 40.02689, -105.28585 40.0...","[POLYGON ((-105.285845 40.02688199999999, -105...","MULTIPOLYGON (((-105.28586 40.02630, -105.2859..."


In [243]:
a_merge = a_merge[['new_geom']].rename(columns={'new_geom': 'geometry'}).set_geometry('geometry', crs=4326)
a_merge.head()

,geometry
0,"MULTIPOLYGON (((-105.50470 39.96170, -105.5046..."
2,"POLYGON ((-105.29503 40.04274, -105.29504 40.0..."
5,"POLYGON ((-105.22584 39.92512, -105.22596 39.9..."
9,"POLYGON ((-105.19578 39.99881, -105.19585 39.9..."
11,"MULTIPOLYGON (((-105.28586 40.02630, -105.2859..."


In [250]:
a_merge.reset_index(inplace=True)

,geometry
0,"MULTIPOLYGON (((-105.50470 39.96170, -105.5046..."
2,"POLYGON ((-105.29503 40.04274, -105.29504 40.0..."
5,"POLYGON ((-105.22584 39.92512, -105.22596 39.9..."
9,"POLYGON ((-105.19578 39.99881, -105.19585 39.9..."
11,"MULTIPOLYGON (((-105.28586 40.02630, -105.2859..."
14,"POLYGON ((-105.16841 40.17369, -105.16870 40.1..."
16,"POLYGON ((-105.28440 40.22712, -105.28456 40.2..."
19,"POLYGON ((-105.17343 39.94821, -105.17343 39.9..."
24,"POLYGON ((-105.17956 40.13560, -105.17935 40.1..."


In [246]:
a.loc[~a.index.isin(a_merge.index)].index

Int64Index([1, 3, 4, 6, 7, 8, 10, 12, 13, 15, 17, 18, 20, 21, 22, 23, 25, 26], dtype='int64')

In [247]:
# Concat to orginal Azone dataframe
a = a.loc[~a.index.isin(a_merge.index)]
a = pd.concat([a, a_merge])

In [251]:
a.reset_index(inplace=True)

In [252]:
a.to_file('a_merged.shp')